<a href="https://colab.research.google.com/github/shivampip/MyYOLO/blob/master/TrainingYolo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Training YOLO

### Training Data Creation

#### Collecting Images

* Decide the classes to be detected.
* Download/ collect atleast 100 images of each class

##### Automatic download images

* Install google_image_download
```
pip install google_images_download
```

* Download the images
```
googleimagesdownload --keywords "Polar bears, baloons, Beaches" --format jpg --limit 100
```

* Put all downloaded images in single folder say `imgs`

#### Annotate the images

* Clone this repo
```
git clone https://github.com/tzutalin/labelImg
cd labelImg
```

* Run labelImg
```
python labelImg.py
```

* Select open dir as `imgs`
* Select destination dir as `imgs`
* Select format as YOLO
* Start Annotating
* Use shortcut `d` for next image and `w` to create box
* **Keep Calm and Annotate**

#### Packing the images

* Now you will have a folder named `imgs`
* In that folder you will have all the images and a `.txt` file corresponding to each image file
* It will look something like this
```
imgs
  |--photo1.jpg
  |--photo1.txt
  |--photo2.jpg
  |--photo2.txt
  ......
  ......
```

* Now `zip` the `imgs` folder to `imgs.zip`

**Training Data Creating Part Completed**

### Clone repo

In [0]:
! git clone https://github.com/AlexeyAB/darknet.git

In [0]:
cd darknet

### Enable GPU and OpenCV and Install

In [0]:
text= ""
with open("Makefile", 'r') as mf:
    text= mf.read().replace("GPU=0", "GPU=1")
    text= text.replace("OPENCV=0", "OPENCV=1")
with open("Makefile", "w") as f:
    f.write(text)

In [0]:
! make

### Download Pre trained weights

In [0]:
! wget https://pjreddie.com/media/files/yolov3.weights

In [0]:
! wget https://pjreddie.com/media/files/darknet19_448.conv.23

### Testing

In [0]:
! ./darknet detect ./cfg/yolov3.cfg ./yolov3.weights ./data/person.jpg .

In [0]:
from matplotlib import pyplot as plt
plt.imshow(plt.imread("predictions.jpg"))

### Preparing Training Data

#### Uploading Annotated Images

* Upload `imgs.zip` file here by running the bilow cell

In [0]:
from google.colab import files
files.upload()

In [0]:
! unzip imgs.zip -d data/

In [0]:
! rm -rf imgs.zip

#### Creating train.txt and test.txt

In [0]:
import os

all_files= os.listdir("data/imgs/")
total_num= len(all_files)

all_files= [fname for fname in all_files if any(ext in fname for ext in ['.jpg', '.png', 'jpeg'])]
all_files= ['data/imgs/'+name for name in all_files]

test_size= 0.1

test_num= round(total_num * test_size)

test_files= all_files[:test_num]
train_files= all_files[test_num:]

trainf_data= ""
testf_data= ""
for fname in test_files:
    testf_data+= fname+"\n"
    
for fname in train_files:
    trainf_data+= fname+"\n"

with open("train.txt", "w") as trainf:
    trainf.write(trainf_data)
    
with open("test.txt", "w") as testf:
    testf.write(testf_data)

#### User Input

In [0]:
classes= ['doraemon', 'nobita']

batch=64        # this means we will be using 64 images for every training step
subdivisions=8  # the batch will be divided by 8 to decrease GPU VRAM requirements (decrease if GPU is powerful)
num_classes= len(classes)
filters= (num_classes+ 5)*5

#### Preparing yolo-voc.cfg file

In [0]:
! wget https://raw.githubusercontent.com/shivampip/MyYOLO/master/files/yolo-voc.cfg

In [0]:
text= ""
with open("yolo-voc.cfg", 'r') as mf:
    text= mf.read().replace("$$BATCH_SIZE$$", str(batch))
    text= text.replace("$$SUBDIVISION$$", str(subdivisions))
    text= text.replace("$$CLASSES$$", str(num_classes))
    text= text.replace("$$FILTERS$$", str(filters))
    
with open("yolo-voc.cfg", "w") as f:
    f.write(text)

#### Creating obj.names file

In [0]:
text= ""
for clas in classes:
    text+=clas+"\n"

with open("cfg/obj.names", "w") as f:
    f.write(text)

In [0]:
! cat cfg/obj.names

#### Creating obj.data file

In [0]:
text= """classes= {}  
train  = train.txt  
valid  = test.txt  
names = obj.names  
backup = backup/ 
""".format(len(classes))

with open("cfg/obj.data", "w") as f:
    f.write(text)

In [0]:
! cat cfg/obj.data

classes= 2  
train  = train.txt  
valid  = test.txt  
names = obj.names  
backup = backup/ 


### Train

#### Fresh Training

In [0]:
! ./darknet detector train ./cfg/obj.data yolo-voc.cfg darknet19_448.conv.23 -dont_show

#### Continue Training ( after atleast 100 steps)

In [0]:
! ./darknet detector train ./cfg/obj.data yolo-voc.cfg ./backup/yolo-voc_last.weights -dont_show

### Test

In [0]:
! ./darknet detector test ./cfg/obj.data yolo-voc.cfg ./backup/yolo-voc_last.weights data/obj/pos-207.jpg -dont_show

In [0]:
plt.imshow(plt.imread("predictions.jpg"))

### Save trained model in Google Drive

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
cp ./darknet/backup/yolo-voc_last.weights ./drive/'My Drive'/ColabFiles
#cp ./darknet/backup/yolo-voc_1000.weights ./drive/'My Drive'/ColabFiles

### Temp

In [9]:
from google.colab import files
fname= files.upload()
print(type(fname))
print(list(fname.keys())[0])
name, _ = fname.items()
print(name)

Saving dora5.png to dora5.png
<class 'dict'>
dora5.png


ValueError: ignored